In [41]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Image, display
from urllib.parse import urljoin

section_dict = {'최신뉴스':100,'정치':200,'경제':300,'사회':400,'생활/문화':500,'세계':600,'IT/과학':700}


def print_news(section):
    sid = section_dict[section] 
    url = f'https://news.nate.com/recent?mid=n0{sid}'
    print(f'{url} {section} 뉴스')

    req_header = {
        'referer': url,
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }

    res = requests.get(url,headers = req_header)


    if res.ok:
        html = res.text
        soup = BeautifulSoup(html, "html.parser")

        a_tags = soup.select("div.mlt01 a[href*='view'].lt1")

        for a_tag in a_tags:
            if a_tag.text:
                link = a_tag['href']	
                title = a_tag.text.strip()
                img_tag = a_tag.select_one('span.ib img')
                if img_tag:
                    img_url = urljoin(url,img_tag['src'])
                    display(Image(url=img_url))                
                print(f'https:{link}')
                print(f'{title}\n')


                

In [ ]:
print_news('정치')

In [1]:
import requests
from bs4 import BeautifulSoup
import os

def download_one_episode(title,no,url):
    req_header = {
        'referer' : url,
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }
    imgdir_name = os.path.join('img',title,str(no))
    os.makedirs(imgdir_name, exist_ok=True)

    res = requests.get(url)
    if not res.ok:
        print(f'Error Code = {res.status_code}')
        exit()

    soup = BeautifulSoup(res.text, 'html.parser')
    img_url_list = [img_tag['src'] for img_tag in soup.select("img[src*='IMAG01']")]

    for img_url in img_url_list:
        res = requests.get(img_url, headers=req_header)
        if res.ok:
            img_data = res.content
            file_path = os.path.join(imgdir_name, os.path.basename(img_url))
            with open(file_path, 'wb') as file:
                print(f'Writing to {file_path} ({len(img_data):,} bytes)')
                file.write(img_data)
        else:
            print(f'Error Code = {res.status_code} for {img_url}')

In [ ]:
download_one_episode('이직로그','10화','https://comic.naver.com/webtoon/detail?titleId=837659&no=10&week=mon')